In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problem Statement

A large company named XYZ, employs, at any given point of time, around 4000 employees. However, every year, around 15% of its employees leave the company and need to be replaced with the talent pool available in the job market. The management believes that this level of attrition (employees leaving, either on their own or because they got fired) is bad for the company, because of the following reasons -


-  The former employees’ projects get delayed, which makes it difficult to meet timelines, resulting in a reputation loss among consumers and partners
- A sizeable department has to be maintained, for the purposes of recruiting new talent
- More often than not, the new employees have to be trained for the job and/or given time to acclimatise themselves to the company

Hence, the management has contracted an HR analytics firm to understand what factors they should focus on, in order to curb attrition. In other words, they want to know what changes they should make to their workplace, in order to get most of their employees to stay. Also, they want to know which of these variables is most important and needs to be addressed right away.


Since you are one of the star analysts at the firm, this project has been given to you.

# Goal of the case study

You are required to model the probability of attrition using a logistic regression. The results thus obtained will be used by the management to understand what changes they should make to their workplace, in order to get most of their employees to stay.

# Step 1: Importing and Merging Data

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np, seaborn as sns,matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
in_time = pd.read_csv('/kaggle/input/hr-analytics-case-study/in_time.csv')
manager_survey_data = pd.read_csv('/kaggle/input/hr-analytics-case-study/manager_survey_data.csv')
employee_survey_data = pd.read_csv('/kaggle/input/hr-analytics-case-study/employee_survey_data.csv')
data_dictionary= pd.read_excel('/kaggle/input/hr-analytics-case-study/data_dictionary.xlsx')
out_time = pd.read_csv('/kaggle/input/hr-analytics-case-study/out_time.csv')
general_data = pd.read_csv('/kaggle/input/hr-analytics-case-study/general_data.csv')


In [ ]:
in_time.head()

In [ ]:
in_time.shape

In [ ]:
in_time=in_time.replace(np.nan,0)
in_time.head()

In [ ]:

in_time.iloc[:, 1:] = in_time.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

In [ ]:
in_time.head()

In [ ]:
out_time=out_time.replace(np.nan,0)
out_time.head()

In [ ]:
out_time.iloc[:, 1:] = out_time.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

In [ ]:
out_time.head()

In [ ]:
in_time=in_time.append(out_time)

In [ ]:
in_time.head()

In [ ]:
in_time=in_time.diff(periods=4410)
in_time=in_time.iloc[4410:]
in_time.reset_index(inplace=True)
in_time.head()

In [ ]:
in_time.drop(columns=['index','Unnamed: 0'],axis=1,inplace=True)
in_time.head()

In [ ]:
in_time.shape

In [ ]:
in_time.drop(['2015-01-01', '2015-01-14','2015-01-26','2015-03-05',
             '2015-05-01','2015-07-17','2015-09-17','2015-10-02',
              '2015-11-09','2015-11-10','2015-11-11','2015-12-25'
             ], axis = 1,inplace=True) 

In [ ]:
in_time.head()

In [ ]:
in_time['Actual Time']=in_time.mean(axis=1)

In [ ]:
in_time['Actual Time'].head()

In [ ]:
in_time['hrs']=in_time['Actual Time']/np.timedelta64(1, 'h')
in_time.head()

In [ ]:
in_time.reset_index(inplace=True)
in_time.head()

In [ ]:
in_time.drop(in_time.columns.difference(['index','hrs']), 1, inplace=True)

In [ ]:
in_time.rename(columns={'index': 'EmployeeID'},inplace=True)
in_time.head()

In [ ]:
general_data.head()

In [ ]:
employee_survey_data.head()

# Combining all data files into one consolidated dataframe

In [ ]:
df_1 = pd.merge(employee_survey_data, general_data, how='inner', on='EmployeeID')
hr = pd.merge(manager_survey_data, df_1, how='inner', on='EmployeeID')
hr = pd.merge(in_time, hr, how='inner', on='EmployeeID')
hr.head()

In [ ]:
hr.describe()

# Correcting Datatype for the variable

In [ ]:
hr['JobLevel']=hr['JobLevel'].astype('object')

# Decoding Values

In [ ]:
hr['Education'] = hr['Education'].replace({ 1 : 'Below College', 2: 'College',3: 'Bachelor',4: 'Master',5 : 'Doctor'})
hr['EnvironmentSatisfaction'] = hr['EnvironmentSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hr['JobInvolvement'] = hr['JobInvolvement'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hr['JobSatisfaction'] = hr['JobSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
#hr['RelationshipSatisfaction'] = hr['RelationshipSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',
#                   3: 'High',4: 'Very High'})
hr['PerformanceRating'] = hr['PerformanceRating'].replace({ 1 : 'Low', 2: 'Good',3: 'Excellent',4: 'Outstanding'})
hr['WorkLifeBalance'] = hr['WorkLifeBalance'].replace({ 1 : 'Bad', 2: 'Good',3: 'Better',4: 'Best'})

In [ ]:
hr.head()

In [ ]:
hr['EmployeeCount'].value_counts(ascending=False)

In [ ]:
hr['Over18'].value_counts(ascending=False)

In [ ]:
hr['StandardHours'].value_counts(ascending=False)

# Drop Non Required Columns

In [ ]:
hr.drop(['EmployeeID', 'EmployeeCount','StandardHours','Over18'], axis = 1,inplace=True) 

# Step 2: Inspecting the Dataframe

In [ ]:
# Let's see the head of our master dataset
hr.head()

In [ ]:
# Let's check the dimensions of the dataframe
hr.shape

In [ ]:
# let's look at the statistical aspects of the dataframe
hr.describe()

In [ ]:
# Let's see the type of each column
hr.info()

In [ ]:
hr['EnvironmentSatisfaction'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='EnvironmentSatisfaction',data=hr);

- With analysis, we found mean & median for EnvironmentSatisfaction field is 2.72 & 3, it needs to be whole number hence, accepted is  3.
- From data_dictionary, we know 3 means High.  

Compute missing values

In [ ]:
hr['EnvironmentSatisfaction'] = hr['EnvironmentSatisfaction'].fillna('High')
hr['EnvironmentSatisfaction'].isnull().sum()

In [ ]:
hr['JobSatisfaction'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='JobSatisfaction',data=hr);

- With analysis, we found mean & median for JobSatisfaction field is 2.72 & 3, it needs to be whole number hence, accepted is  3.
- From data_dictionary, we know 3 means High.

Compute missing values

In [ ]:
hr['JobSatisfaction'] = hr['JobSatisfaction'].fillna('High')
hr['JobSatisfaction'].isnull().sum()

In [ ]:
hr['WorkLifeBalance'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='WorkLifeBalance',data=hr);

- With analysis, we found mean & median for WorkLifeBalance field is 2.76 & 3, it needs to be whole number hence, accepted is  3.
- From data_dictionary, we know 3 means Better.

#Compute missing values

In [ ]:
hr['WorkLifeBalance'] = hr['WorkLifeBalance'].fillna('Better')
hr['WorkLifeBalance'].isnull().sum()

In [ ]:
hr['NumCompaniesWorked'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='NumCompaniesWorked',data=hr);

In [ ]:
sns.boxplot(x='NumCompaniesWorked',data=hr);

- With analysis, we found mean & median for NumCompaniesWorked field is 2.69 & 2, it needs to be whole number along with handling outliers hence, accepted is  2.


Compute missing values

In [ ]:
hr['NumCompaniesWorked'] = hr['NumCompaniesWorked'].fillna(2)
hr['NumCompaniesWorked'].isnull().sum()

In [ ]:
hr['TotalWorkingYears'].value_counts(ascending=False)

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.distplot(hr['TotalWorkingYears'], hist=True, kde=False, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('# of Employees')
ax.set_xlabel('TotalWorkingYears');


In [ ]:
sns.boxplot(x='TotalWorkingYears',data=hr);

- With analysis, we found mean & median for TotalWorkingYears field is 11.27 & 10, it needs to be whole number along with handling outliers hence, accepted is  10.

In [ ]:
hr['TotalWorkingYears'] = hr['TotalWorkingYears'].fillna(2)
hr['TotalWorkingYears'].isnull().sum()

In [ ]:
hr.info()

# EDA

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='WorkLifeBalance',data=hr,hue="Attrition")
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

# Insights
- Attrition : Whether the employee left in the previous year or not
1. Employee who left in the previous year are 17% of population (1019) i.e. 174 who believe WorkLifeBalance is Good in org.
2. Employee who left in the previous year are 18% of population(454) i.e. 81 who believe WorkLifeBalance is Best in org.
3. Employee who left in the previous year are 34% of population (239) i.e. 81 who believe WorkLifeBalance is Bad in org.
4. Employee who left in the previous year are 14% of population (2698) i.e. 378 who believe WorkLifeBalance is Better in org.
- People who left in the previous year & believe WorkLifeBalance is Better in org were 52% of population who left in the previous year, second by 24% people who left in the previous year & believe WorkLifeBalance is Good in org

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='PerformanceRating', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

# Insights
- Attrition : Whether the employee left in the previous year or not
1. Employee who left in the previous year are 16% of population (3732) i.e. 597 whose PerformanceRating was Excellent in org.
2.  Employee who left in the previous year are 18% of population (678) i.e. 122 whose PerformanceRating was Outstanding in org.
- People who left in the previous year & PerformanceRating was Excellent in org were 83% of population who left in the previous year.

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='EnvironmentSatisfaction', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

# Insights
- Attrition : Whether the employee left in the previous year or not
1. Employee who left in the previous year are 14% of population (1375) i.e. 192 who believe EnvironmentSatisfaction is High in org. in org.
1. Employee who left in the previous year are 15% of population (856) i.e. 129 who believe EnvironmentSatisfaction is Medium in org. 
1. Employee who left in the previous year are 13% of population (1334) i.e. 173 who believe EnvironmentSatisfaction is Very High in org.
1. Employee who left in the previous year are 25% of population (845) i.e. 211 who believe EnvironmentSatisfaction is Low in org. 
- People who left in the previous year & believe EnvironmentSatisfaction is Low in org were 30% of population who left in the previous year. Second by People who left in the previous year & believe EnvironmentSatisfaction is High in org

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='JobSatisfaction', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='WorkLifeBalance', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='Age',x='Attrition',data=hr)
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='BusinessTravel', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='Department', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='DistanceFromHome',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='Education', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.countplot(x='EducationField', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='Gender', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='JobLevel', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.countplot(x='JobRole', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='MaritalStatus', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='MonthlyIncome',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='PercentSalaryHike',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='StockOptionLevel', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:

plt.figure(figsize=(8,8))
sns.violinplot(y='TotalWorkingYears',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='TrainingTimesLastYear',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='YearsAtCompany',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='YearsSinceLastPromotion',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='YearsWithCurrManager',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(y='hrs',x='Attrition',data=hr)

plt.show()

In [ ]:
plt.figure(figsize=(20,18))
sns.heatmap(hr.corr(), annot = True, cmap="Accent");

In [ ]:
hr_num=hr[[ 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','hrs']]

sns.pairplot(hr_num, diag_kind='kde')
plt.show()

For categorical variables with multiple levels, create dummy features (one-hot encoded)

In [ ]:
hr.info()

In [ ]:
hr.columns

In [ ]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy1 = pd.get_dummies(hr[['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
                                 'JobSatisfaction', 'WorkLifeBalance','BusinessTravel', 'Department',
                                 'Education','EducationField', 'Gender', 'JobLevel', 'JobRole',
                                 'MaritalStatus']], drop_first=True)

# Adding the results to the master dataframe
hr = pd.concat([hr, dummy1], axis=1)

In [ ]:
hr.head()

In [ ]:
hr.shape

# Dropping the repeated variables

In [ ]:
# We have created dummies for the below variables, so we can drop them
hr = hr.drop(['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
                                 'JobSatisfaction', 'WorkLifeBalance','BusinessTravel', 'Department',
                                 'Education','EducationField', 'Gender', 'JobLevel', 'JobRole',
                                 'MaritalStatus'], 1)

In [ ]:
hr.head()

In [ ]:
hr.shape

# Mapping Attrition to 1/0 

In [ ]:
hr['Attrition'] = hr['Attrition'].replace({'Yes': 1, "No": 0})

In [ ]:
hr.head()

In [ ]:
hr.shape

 # Step 3: Test-Train Split

In [ ]:
from sklearn.model_selection import train_test_split
# Putting feature variable to X
X = hr.drop(['Attrition'], axis=1)

X.head()

In [ ]:
X.shape

In [ ]:
# Putting response variable to y
y = hr['Attrition']

y.head()

In [ ]:
y.shape

In [ ]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)


In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
 X_test.head()

In [ ]:
 X_test.shape

In [ ]:
y_train.head()

In [ ]:
y_train.shape

In [ ]:
y_test.head()

In [ ]:
y_test.shape

# Step 5: Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[[ 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','hrs']] = scaler.fit_transform(X_train[[ 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','hrs']])

X_train.head()

In [ ]:
### Checking the Attrition Rate
Attrition = (sum(hr['Attrition'])/len(hr['Attrition'].index))*100
Attrition

We have almost 16.13% Attrition rate

# Step 6: Looking at Correlations

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (50,40))   
sns.heatmap(hr.corr(),annot = True,cmap="tab20c")
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
hr.corr()['Attrition'].sort_values(ascending = False).plot(kind='bar');

Dropping highly correlated dummy variables

In [ ]:
corrmat = X_train.corr() 
corrdf = corrmat.where(np.triu(np.ones(corrmat.shape), k=1).astype(np.bool))
corrdf = corrdf.unstack().reset_index()
corrdf.columns = ['Var1', 'Var2', 'Correlation']
corrdf.dropna(subset = ['Correlation'], inplace = True)
corrdf['Correlation'] = round(corrdf['Correlation'], 2)
corrdf['Correlation'] = abs(corrdf['Correlation'])
matrix= corrdf.sort_values(by = 'Correlation', ascending = False).head(50)
matrix

In [ ]:
unique=list(set(matrix.Var2))
len(unique)

Dropping highly correlated dummy variables

In [ ]:
X_test = X_test.drop(unique,1)
X_train = X_train.drop(unique,1)

In [ ]:
X_test.shape

In [ ]:
X_test.head() 

In [ ]:
X_train.head()

In [ ]:
X_train.shape

### Checking the Correlation Matrix

After dropping highly correlated variables now let's check the correlation matrix again.

In [ ]:
plt.figure(figsize = (50,25))
sns.heatmap(X_train.corr(),annot = True,cmap="tab20c")
plt.show()

 # Step 7: Model Building


Let's start by splitting our data into a training set and a test set.
- Running Your First Training Model

In [ ]:
import statsmodels.api as sm
# Logistic regression model
logm1 = sm.Logit(y_train,(sm.add_constant(X_train)))
logm1.fit().summary()

# Step 8: Feature Selection Using RFE

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
from sklearn.feature_selection import RFE
rfe = RFE(logreg, 13)             
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]


Assessing the model with StatsModels

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.Logit(y_train,X_train_sm)
res = logm2.fit()
res.summary()

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred[:10]

In [ ]:
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred[:10]


In [ ]:
y_train_pred_final = pd.DataFrame({'Attrition':y_train.values, 'Attrition_Prob':y_train_pred})
y_train_pred_final['EmployeeID'] = y_train.index
y_train_pred_final.head()

Creating new column 'predicted' with 1 if Attrition_Prob > 0.5 else 0

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
from sklearn import metrics
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Attrition, y_train_pred_final.predicted )
print(confusion)

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

# Checking VIFs

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

There are a few variables with high p value . It's best to drop these variables as they aren't helping much with prediction and unnecessarily making the model complex. The variable 'JobRole_Human Resources' has the highest p value. So let's start by dropping that.

In [ ]:
col = col.drop('JobRole_Human Resources', 1)
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm3 = sm.Logit(y_train,X_train_sm)
res = logm3.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Attrition_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

So overall the accuracy hasn't dropped much.

Let's check the VIFs again

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop Education_Doctor since it has a high VIF
col = col.drop('Education_Doctor')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm4 = sm.Logit(y_train,X_train_sm)
res = logm4.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))


So overall the accuracy hasn't dropped much.

Let's check the VIFs again

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop EducationField_Other since it has a high p value
col = col.drop('EducationField_Other')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm5 = sm.Logit(y_train,X_train_sm)
res = logm5.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

The accuracy is still practically the same.

In [ ]:
#Let's now check the VIFs again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop EducationField_Technical Degree since it has a high p value
col = col.drop('EducationField_Technical Degree')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm6 = sm.Logit(y_train,X_train_sm)
res = logm6.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

So overall the accuracy hasn't dropped much.

In [ ]:
#Let's now check the VIFs again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop JobInvolvement_Very High since it has a high p value
col = col.drop('JobInvolvement_Very High')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm7 = sm.Logit(y_train,X_train_sm)
res = logm7.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

The accuracy is still practically the same.

In [ ]:
#Let's now check the VIFs again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop JobInvolvement_Low since it has a high p value
col = col.drop('JobInvolvement_Low')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm8 = sm.Logit(y_train,X_train_sm)
res = logm8.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

So overall the accuracy hasn't dropped much.

In [ ]:
#Let's now check the VIFs again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Let's drop JobLevel_5 since it has a high p value
col = col.drop('JobLevel_5')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm9 = sm.Logit(y_train,X_train_sm)
res = logm9.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final['Attrition_Prob'] = y_train_pred
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.predicted))

So overall the accuracy hasn't dropped much.

In [ ]:
#Let's now check the VIFs again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

All variables have a good value of VIF. So we need not drop any more variables and we can proceed with making predictions using this model only

# Metrics beyond simply accuracy

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)


In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - 
print(FP/ float(TN+FP))

In [ ]:
# positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

# Step 9: Plotting the ROC Curve

An ROC curve demonstrates several things:

* It shows the tradeoff between sensitivity and specificity (any increase in sensitivity will be accompanied by a decrease in specificity).
* The closer the curve follows the left-hand border and then the top border of the ROC space, the more accurate the test.
* The closer the curve comes to the 45-degree diagonal of the ROC space, the less accurate the test.


In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Attrition, y_train_pred_final.Attrition_Prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.Attrition, y_train_pred_final.Attrition_Prob)

# Step 10: Finding Optimal Cutoff Point

Optimal cutoff probability is that prob where we get balanced sensitivity and specificity

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Attrition_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Attrition, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
ax= cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
ax.set_axisbelow(True)
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

From the curve above, 0.18 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Attrition_Prob.map( lambda x: 1 if x > 0.18 else 0)

y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Attrition, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.Attrition, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)


In [ ]:
# Calculate false postive rate 
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

# Precision and Recall

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_pred_final.Attrition, y_train_pred_final.predicted)

In [ ]:
recall_score(y_train_pred_final.Attrition, y_train_pred_final.predicted)

# Precision and recall tradeoff

In [ ]:
from sklearn.metrics import precision_recall_curve
y_train_pred_final.Attrition.head()

In [ ]:
 y_train_pred_final.predicted.head()

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Attrition, y_train_pred_final.Attrition_Prob)

In [ ]:
plt.plot(thresholds, p[:-1], "g-")
plt.plot(thresholds, r[:-1], "r-")
plt.show()

# Making predictions on the test set

In [ ]:
X_test = X_test[col]
X_test.head()

In [ ]:
X_test_sm = sm.add_constant(X_test)

In [ ]:
y_test_pred = res.predict(X_test_sm)

In [ ]:
y_test_pred[:10]

In [ ]:
# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)

In [ ]:
# Let's see the head
y_pred_1.head()

In [ ]:
# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)

In [ ]:
# Putting EmployeeID to index
y_test_df['EmployeeID'] = y_test_df.index

In [ ]:
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

In [ ]:
# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
# Renaming the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Attrition_Prob'})

In [ ]:
# Rearranging the columns
y_pred_final = y_pred_final.reindex(['EmployeeID','Attrition','Attrition_Prob'], axis=1)

In [ ]:
# Let's see the head of y_pred_final
y_pred_final.head()

In [ ]:
y_pred_final['final_predicted'] = y_pred_final.Attrition_Prob.map(lambda x: 1 if x > 0.18 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_pred_final.Attrition, y_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_pred_final.Attrition, y_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)